In [317]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import diffprivlib.models as dp
from sklearn.linear_model import LogisticRegression
from torch import nn, optim
from opacus import PrivacyEngine
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader


# Importing train, test split library
from sklearn.model_selection import train_test_split

# efficacy metrics from sklearn
from sklearn import metrics


In [318]:
# bias
# holisticai imports
import holisticai
from holisticai.bias.metrics import classification_bias_metrics
from holisticai.bias.mitigation.postprocessing import CalibratedEqualizedOdds, RejectOptionClassification

## Load Training and Testing Data

In [319]:
# # Load the data
# X_train = pd.read_csv('../../data/X_train.csv',index_col=0).drop(['id'],axis=1)
# y_train = pd.read_csv('../../data/y_train.csv',index_col=0)
# X_test = pd.read_csv('../../data/X_test.csv',index_col=0).drop(['id'],axis=1)
# y_test = pd.read_csv('../../data/y_test.csv',index_col=0)

In [320]:
baseline__df = pd.read_csv("../../data/data_baseline.csv",index_col=0)
baseline__df.head()

,sex,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_desc,two_year_recid,time_in_custody_in_days,time_in_jail_in_days,African-American_race,Asian_race,Hispanic_race,Native American_race,Other_race,Felony_c_charge_degree,High_score_text,Low_score_text,High_v_score_text
id,,,,,,,,,,,,,,,,,,,
1,1,69,0,0,0,0,0.285811,0,7.0,0.0,0,0,0,0,1,1,0,1,0
3,1,34,0,0,0,0,0.563569,1,10.0,10.0,1,0,0,0,0,1,0,1,0
4,1,24,0,0,1,4,0.594859,1,0.0,1.0,1,0,0,0,0,1,0,1,0
5,1,23,0,1,0,1,0.375455,0,0.0,0.0,1,0,0,0,0,1,1,0,0
6,1,43,0,0,0,2,0.448707,0,0.0,0.0,0,0,0,0,1,1,0,1,0


In [321]:
baseline__df.columns

Index(['sex', 'age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count',
       'priors_count', 'c_charge_desc', 'two_year_recid',
       'time_in_custody_in_days', 'time_in_jail_in_days',
       'African-American_race', 'Asian_race', 'Hispanic_race',
       'Native American_race', 'Other_race', 'Felony_c_charge_degree',
       'High_score_text', 'Low_score_text', 'High_v_score_text'],
      dtype='object')

In [322]:
#baseline__df = baseline__df.drop(['Asian_race','Hispanic_race','Other_race','High_score_text','High_v_score_text','Low_score_text','Native American_race'],axis=1)

In [323]:
baseline__df.columns

Index(['sex', 'age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count',
       'priors_count', 'c_charge_desc', 'two_year_recid',
       'time_in_custody_in_days', 'time_in_jail_in_days',
       'African-American_race', 'Felony_c_charge_degree'],
      dtype='object')

In [324]:
X_train, X_test, y_train, y_test = train_test_split(
    baseline__df.drop(["two_year_recid"], axis=1),
    baseline__df["two_year_recid"],
    test_size=0.2,
    random_state=1000,
    stratify=baseline__df["two_year_recid"],
)

In [325]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [326]:
y_train.head()

,two_year_recid
id,
8193,1
5507,0
39,1
3692,1
8602,1


## Transform Data for the Model

In [327]:
type(X_train)

pandas.core.frame.DataFrame

In [328]:
# Check type of y_test
type(y_test)

pandas.core.frame.DataFrame

In [329]:
y_test.head()

,two_year_recid
id,
5519,0
6466,0
1369,0
8179,1
8696,0


In [330]:
# group_a_train = X_train["African-American_race"] == 1
# group_b_train = X_train["African-American_race"] == 0
# data_train = [X_train, y_train, group_a_train, group_b_train]


# group_a_test = X_test["African-American_race"] == 1
# group_b_test = X_test["African-American_race"] == 0
# data_test = [X_test, y_test, group_a_test, group_b_test]


In [331]:
group_a_train = X_train["sex"] == 1
group_b_train = X_train["sex"] == 0
data_train = [X_train, y_train, group_a_train, group_b_train]


group_a_test = X_test["sex"] == 1
group_b_test = X_test["sex"] == 0
data_test = [X_test, y_test, group_a_test, group_b_test]


Defining the functions to calcluate the perfomace metrics

In [332]:

# dictionnary of metrics
metrics_dict={
        "Accuracy": metrics.accuracy_score,
        "Balanced accuracy": metrics.balanced_accuracy_score,
        "Precision": metrics.precision_score,
        "Recall": metrics.recall_score,
        "F1-Score": metrics.f1_score}

# efficacy metrics dataframe helper tool
def metrics_dataframe(y_pred, y_true, metrics_dict=metrics_dict):
    metric_list = [[pf, fn(y_true, y_pred)] for pf, fn in metrics_dict.items()]
    return pd.DataFrame(metric_list, columns=["Metric", "Value"]).set_index("Metric")

In [333]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [334]:
# Check the shapes of the scaled data
X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape

((5748, 11), (1437, 11), (5748, 1), (1437, 1))

Checking Prediction distribution in training and test data

In [335]:
test_df = X_test.copy()
test_df['y_test'] = y_test

In [336]:
test_proportion = test_df[['African-American_race','y_test']]
test_proportion = test_proportion.groupby(['African-American_race']).aggregate(['sum','count'])
test_proportion['prop_of_recidivism'] = test_proportion['y_test']['sum']/test_proportion['y_test']['count']
test_proportion

y_test       prop_of_recidivism
                         sum count                   
African-American_race                                
0                        269   707           0.380481
1                        378   730           0.517808

### 1. Baseline Logistic Regression Model

In [337]:
baseline_lr = LogisticRegression(solver="lbfgs", max_iter=1000)
baseline_lr.fit(X_train_scaled, y_train)

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [338]:
y_pred = baseline_lr.predict(X_test_scaled)
y_proba = baseline_lr.predict_proba(X_test_scaled)
y_score = y_proba[:,1]
y_true = y_test

Model Performance

In [339]:
# Baseline efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.691719
Balanced accuracy,0.683110
Precision,0.679577
Recall,0.596600
F1-Score,0.635391


Fairness Performance

In [340]:
fairness_metrics = classification_bias_metrics(group_a_test, group_b_test, y_pred, y_true, metric_type='both')
fairness_metrics.iloc[6]

Value        0.165842
Reference    0.000000
Name: False Positive Rate Difference, dtype: float64

In [341]:
test_df['y_pred_lrb'] = y_pred
test_df['y_score_lrb'] = y_score

In [342]:
test_proportion = test_df[['African-American_race','y_pred_lrb']]
test_proportion = test_proportion.groupby(['African-American_race']).aggregate(['sum','count'])
test_proportion['prop_of_recidivism'] = test_proportion['y_pred_lrb']['sum']/test_proportion['y_pred_lrb']['count']
test_proportion

y_pred_lrb       prop_of_recidivism
                             sum count                   
African-American_race                                    
0                            199   707           0.281471
1                            369   730           0.505479

In [343]:
list(X_train.columns)

['sex',
 'age',
 'juv_fel_count',
 'juv_misd_count',
 'juv_other_count',
 'priors_count',
 'c_charge_desc',
 'time_in_custody_in_days',
 'time_in_jail_in_days',
 'African-American_race',
 'Felony_c_charge_degree']

In [344]:
feature_imp_baseline = pd.DataFrame({'Varibale' : list(X_train.columns), 'coeff' : list(baseline_lr.coef_[0])})
feature_imp_baseline.sort_values(by='coeff',ascending=False)

,Varibale,coeff
6,c_charge_desc,0.726139
5,priors_count,0.621969
8,time_in_jail_in_days,0.188708
4,juv_other_count,0.133201
7,time_in_custody_in_days,0.113818
0,sex,0.101987
2,juv_fel_count,0.101817
9,African-American_race,0.027199
3,juv_misd_count,0.004828
10,Felony_c_charge_degree,-0.133275


### 2. Differentially Private Logistic Regression Model


epsilon (ε) represents the maximum acceptable difference in the model's output when the dataset is modified by adding or removing a single data point. 
Intuition: 
When epsilon is provided to a logistic regression model, it guides the model to learn a representation of the data that is less sensitive to individual changes.

By introducing epsilon, the logistic regression model limits the amount of information that can be inferred about any single individual. This is achieved by adding noise to the model's parameters or predictions, making it difficult for an adversary to discern the impact of any single data point.

This is equivalent to learning Fair Representation in terms of Fairness, whereas individual's association witht the outcome is obfuscated, and therefore we expect to see fairer model just by introducing the privacy budget

To-do : *write a loop plot*

In [374]:
dp_clf = dp.LogisticRegression(random_state=0,epsilon=9)
dp_clf.fit(X_train_scaled, y_train)

c:\Users\athar\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\athar\anaconda3\lib\site-packages\diffprivlib\models\logistic_regression.py:239: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "


LogisticRegression(accountant=BudgetAccountant(spent_budget=[(13, 0), (13, 0), (13, 0), (13, 0), (13, 0), ...]),
                   data_norm=30.38832773881192, epsilon=9, random_state=0)

In [375]:
y_pred_dp = dp_clf.predict(X_test_scaled)
y_proba_dp = dp_clf.predict_proba(X_test_scaled)
y_score_dp = y_proba_dp[:,1]

Private Model Performance

In [376]:
# Baseline efficacy
metrics_dataframe(y_pred_dp, y_true)

,Value
Metric,
Accuracy,0.695894
Balanced accuracy,0.687887
Precision,0.682292
Recall,0.607419
F1-Score,0.642682


Private Model Fairness Performance

In [377]:
fairness_metrics_dp = classification_bias_metrics(group_a_test, group_b_test, y_pred_dp, y_true, metric_type='both')
fairness_metrics_dp.iloc[6]

Value        0.138349
Reference    0.000000
Name: False Positive Rate Difference, dtype: float64

Private Model Fairness Interpretation

In [378]:
feature_imp_dp = pd.DataFrame({'Varibale' : list(X_train.columns), 'coeff' : list(dp_clf.coef_[0])})
feature_imp_dp.sort_values(by='coeff',ascending=False)

,Varibale,coeff
5,priors_count,0.709023
6,c_charge_desc,0.694571
8,time_in_jail_in_days,0.274882
4,juv_other_count,0.122213
2,juv_fel_count,0.065261
7,time_in_custody_in_days,0.024028
0,sex,0.019511
9,African-American_race,-0.008180
3,juv_misd_count,-0.030934
10,Felony_c_charge_degree,-0.098154


In [379]:
print("Privacy Measure : ", dp_clf.epsilon)
print("Fairness measure: ", fairness_metrics_dp.iloc[6][0])

Privacy Measure :  9
Fairness measure:  0.13834895530916763


<h3> Implementing Fairness

1. Caliberated Equalized Odds

In [382]:
# initialize object
ceo = CalibratedEqualizedOdds(cost_constraint="fpr")

In [383]:
# predict train set
y_pred_train_dp = dp_clf.predict(X_train_scaled)
y_proba_train_dp = dp_clf.predict_proba(X_train_scaled)

In [384]:
# fit it
ceo.fit(y_train, y_proba_train_dp, group_a_train, group_b_train)

In [385]:
# transform it
d = ceo.transform(y_test, y_proba_dp, group_a_test, group_b_test, 0.8)
# new predictions
y_pred_ceo = d['y_pred']

Fair Model Perormance : CEO

In [386]:
# efficacy
metrics_dataframe(y_pred_ceo, y_test)

,Value
Metric,
Accuracy,0.604732
Balanced accuracy,0.564548
Precision,0.806202
Recall,0.160742
F1-Score,0.268041


Fair Model Fairness Performance :  CEO

In [387]:
fairness_metrics_ceo = classification_bias_metrics(group_a_test, group_b_test, y_pred_ceo, y_test, metric_type='both')
print("Fairness metric : ", fairness_metrics_ceo.iloc[6][0])

Fairness metric :  0.040783034257748776


c:\Users\athar\anaconda3\lib\site-packages\holisticai\bias\metrics\_classification.py:172: RuntimeWarning: divide by zero encountered in double_scalars
  return sr_a / sr_b
c:\Users\athar\anaconda3\lib\site-packages\holisticai\bias\metrics\_classification.py:223: RuntimeWarning: divide by zero encountered in double_scalars
  return min(sr_a / sr_b, sr_b / sr_a)


In [357]:
#Creating the dataframe for the different values of discrimination threshold


#Looping through the diffent values 
for i in range(0,1,0.1):
	# transform it
	d = ceo.transform(y_test, y_proba_dp, group_a_test, group_b_test, 0.8)
	# new predictions
	y_pred_ceo = d['y_pred']
	


TypeError: 'float' object cannot be interpreted as an integer

Fairness got worse

In [ ]:
feature_imp_dp = pd.DataFrame({'Varibale' : list(X_train.columns), 'coeff' : list(dp_clf.coef_[0])})
feature_imp_dp.sort_values(by='coeff',ascending=False)

<h3> Reject Option Classification

In [388]:
#changing the lables since ROC considers 1 as fav label, and optimizes for it
y_train_flipped = y_train.copy()
y_train_flipped['two_year_recid'] = np.where(y_train_flipped['two_year_recid'] == 1,0,1)

y_test_flipped = y_test.copy()
y_test_flipped['two_year_recid'] = np.where(y_test_flipped['two_year_recid'] == 1,0,1)

fitting new lr model with swapped labels

In [ ]:
# dp_clf2 = dp.LogisticRegression(random_state=0,epsilon=13)
# dp_clf2.fit(X_train_scaled, y_train_flipped)

# # predict train set
# y_pred_train_dp_flipped = dp_clf2.predict(X_train_scaled)
# y_proba_train_dp_flipped = dp_clf2.predict_proba(X_train_scaled)

# # predict test set
# y_pred_test_dp_flipped = dp_clf2.predict(X_test_scaled)
# y_proba_test_dp_flipped = dp_clf2.predict_proba(X_test_scaled)
# y_score_test_dp_flipped = y_proba_test_dp_flipped[:,1]

In [ ]:
# #hold
# y_proba_train_dp_flipped = y_proba_train_dp.copy()
# y_proba_train_dp_flipped = y_proba_train_dp_flipped[:,[1,0]]

# y_proba_dp_flipped = y_proba_dp.copy()
# y_proba_dp_flipped = y_proba_dp_flipped[:,[1,0]]

In [389]:
# initialize
roc = RejectOptionClassification(metric_name="Statistical parity difference")
# fit it
roc.fit(y_train, y_proba_train_dp, group_a_train, group_b_train)

In [390]:
d = roc.transform(y_test, y_proba_dp, group_a_test, group_b_test)
d

{'y_pred': array([False, False, False, ..., False, False, False]),
 'y_score': array([0.21969614, 0.23971222, 0.60155032, ..., 0.51254205, 0.40473922,
        0.49514075])}

In [391]:
# new predictions
y_pred_roc = d['y_pred']
y_pred_roc

array([False, False, False, ..., False, False, False])

In [392]:
# efficacy
metrics_dataframe(y_pred_roc, y_test)

,Value
Metric,
Accuracy,0.605428
Balanced accuracy,0.565321
Precision,0.807692
Recall,0.162287
F1-Score,0.270270


In [393]:
# bias metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred_roc, y_test, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.050259,0
Disparate Impact,2.005186,1
Four Fifths Rule,0.498707,1
Cohen D,0.175616,0
2SD Rule,2.630758,0
Equality of Opportunity Difference,0.077545,0
False Positive Rate Difference,0.011659,0
Average Odds Difference,0.044602,0
Accuracy Difference,-0.059794,0


In [ ]:

test_df['y_pred_dp'] = y_pred_dp
test_df['y_score_dp'] = y_score_dp

test_df['y_pred_roc'] = d['y_pred']
test_df['y_score_roc'] = d['y_score']

In [ ]:
test_df.columns

Index(['sex', 'age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count',
       'priors_count', 'c_charge_desc', 'time_in_custody_in_days',
       'time_in_jail_in_days', 'African-American_race', 'Asian_race',
       'Hispanic_race', 'Native American_race', 'Other_race',
       'Felony_c_charge_degree', 'High_score_text', 'Low_score_text',
       'High_v_score_text', 'y_test', 'y_pred_lrb', 'y_score_lrb', 'y_pred_dp',
       'y_score_dp', 'y_pred_roc', 'y_score_roc'],
      dtype='object')

In [ ]:
test_df[['African-American_race','y_pred_lrb','y_score_lrb']].groupby(['African-American_race']).aggregate(['min','max'])

y_pred_lrb     y_score_lrb          
                             min max         min       max
African-American_race                                     
0                              0   1    0.026461  0.984259
1                              0   1    0.028240  0.984290

In [ ]:
test_df[['African-American_race','y_pred_roc','y_score_roc']].groupby(['African-American_race']).aggregate(['min','max'])

y_pred_roc       y_score_roc          
                             min   max         min       max
African-American_race                                       
0                          False  True    0.022286  0.997275
1                          False  True    0.018326  0.990368

In [ ]:
d_df = pd.DataFrame(d)
d_df.groupby('y_pred').aggregate(['min','max'])

y_score          
             min       max
y_pred                    
False   0.018326  0.860765
True    0.861981  0.997275